In [105]:
#https://autokeras.com/tutorial/timeseries_forecaster/

import pandas as pd
import tensorflow as tf

import autokeras as ak

from datetime import datetime

In [106]:
ab_path = "data/SPY-Daily-Y09-Now.csv"
stockname='SPY'

train_ratio=0.8

# start_date='2015-01-01'
start_date='2022-01-01'
end_date='2022-12-31'

date_col='Date'
prediction_col='EMA1'
feature_cols=['EMA1','EMA2','MACD']

df=pd.read_csv(f'{ab_path}',parse_dates=['Date/Time'],dayfirst=True)
df.rename(columns={'Date/Time':date_col},inplace=True)
df.drop(columns=['Ticker'],inplace=True)
allCols=list(df.columns)
print(allCols)

df[date_col]=df[date_col].apply( lambda  dx :  datetime(dx.year,dx.month,dx.day ))
df.set_index(date_col,inplace=True)
df=df.loc[start_date:end_date,:]
df=df[feature_cols]
print(df.info())
df.tail()

['Date', 'Price', 'EMA1', 'EMA2', 'EMA3', 'MACD', 'SIGNAL', 'RSI', 'RSI-MA']
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2022-01-03 to 2022-12-30
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EMA1    251 non-null    float64
 1   EMA2    251 non-null    float64
 2   MACD    251 non-null    float64
dtypes: float64(3)
memory usage: 7.8 KB
None


,EMA1,EMA2,MACD
Date,,,
2022-12-23,386.73,389.85,-3.11
2022-12-27,385.76,389.04,-3.28
2022-12-28,384.11,387.86,-3.75
2022-12-29,383.99,387.44,-3.45
2022-12-30,383.70,386.97,-3.26


In [107]:
val_split = int(len(df) * train_ratio)
data_train = df[:val_split]
validation_data = df[val_split:]

In [108]:
data_x = data_train[
   list(df.columns)
].astype("float64")

In [109]:
data_x_val = validation_data[
    list(df.columns)
].astype("float64")

In [110]:
data_x_test=df[
     list(df.columns)
].astype("float64")

In [111]:
data_y = data_train[prediction_col].astype("float64")
data_y_val = validation_data[prediction_col].astype("float64")
print(data_x.shape) 
print(data_y.shape) 

(200, 3)
(200,)


In [112]:
print(data_y .shape) 
print(data_y_val.shape) 

(200,)
(51,)


In [116]:
# run suceeded if lookback< predict_until ????
predict_until = 50
lookback = 5

In [117]:
clf = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_until=predict_until,
    max_trials=3,
    objective="val_loss",
)

INFO:tensorflow:Reloading Tuner from .\time_series_forecaster\tuner0.json


INFO:tensorflow:Reloading Tuner from .\time_series_forecaster\tuner0.json


In [118]:
clf.fit(
    x=data_x,
    y=data_y,
    validation_data=(data_x_val, data_y_val),
    batch_size=32,
    epochs=10,
)

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
7/7 [==============================] - 11s 402ms/step - loss: 173006.2031 - mean_squared_error: 173006.2031 - val_loss: 150908.5469 - val_mean_squared_error: 150908.5469
Epoch 2/10
7/7 [==============================] - 0s 24ms/step - loss: 172966.0469 - mean_squared_error: 172966.0469 - val_loss: 150842.1875 - val_mean_squared_error: 150842.1875
Epoch 3/10
7/7 [==============================] - 0s 30ms/step - loss: 172869.3281 - mean_squared_error: 172869.3281 - val_loss: 150773.5938 - val_mean_squared_error: 150773.5938
Epoch 4/10
7/7 [==============================] - 0s 27ms/step - loss: 172824.1875 - mean_squared_error: 172824.1875 - val_loss: 150703.2500 - val_mean_squared_error: 150703.2500
Epoch 5/10
7/7 [==============================] - 0s 34ms/step - loss: 172708.5938 - mean_squared_error: 172708.5938 - val_loss: 150631.0156 - val_mean_squared_error: 150631.0156
Epoch 6/10
7/7 [==============================] - 0s 32ms/step - loss: 172681.4688 - mean_squared_error

INFO:tensorflow:Assets written to: .\time_series_forecaster\best_model\assets


INFO:tensorflow:Assets written to: .\time_series_forecaster\best_model\assets


In [119]:
# Predict with the best model(includes original training data).
predictions = clf.predict(data_x_test)
print(predictions.shape)
# Evaluate the best model with testing data.
print(clf.evaluate(data_x_val, data_y_val))

8/8 [==============================] - 0s 13ms/step
(46, 1)
2/2 [==============================] - 2s 21ms/step - loss: 150242.6562 - mean_squared_error: 150242.6562
[150242.65625, 150242.65625]
